# TripScore — TDX Live Smoke Check

This notebook calls the real TDX APIs and prints counts to confirm credentials + datasets work.

Requirements:
- `TDX_CLIENT_ID` / `TDX_CLIENT_SECRET` configured in the environment
- network access


In [ ]:
import os
import sys
from pathlib import Path

repo_root = Path.cwd()
src_dir = repo_root / 'src'
if src_dir.exists():
    sys.path.insert(0, str(src_dir))

try:
    from tripscore.core.env import load_dotenv_if_present

    load_dotenv_if_present()
except Exception:
    pass

print('TDX_CLIENT_ID configured:', bool(os.getenv('TDX_CLIENT_ID')))
print('TDX_CLIENT_SECRET configured:', bool(os.getenv('TDX_CLIENT_SECRET')))


In [ ]:
from pathlib import Path

from tripscore.config.settings import get_settings
from tripscore.core.cache import FileCache
from tripscore.ingestion.tdx_client import TdxClient

from tripscore.core.env import resolve_project_path

settings = get_settings()
cache_dir = resolve_project_path(settings.cache.dir)
cache = FileCache(cache_dir, enabled=settings.cache.enabled)
tdx = TdxClient(settings, cache)

print('TDX city:', settings.ingestion.tdx.city)
print('Cache dir:', cache_dir)


In [ ]:
# Helpers: schema checks + cache checks
from dataclasses import asdict, is_dataclass


def count_cache_files() -> int:
    base = cache_dir / 'tdx'
    if not base.exists():
        return 0
    return sum(1 for p in base.glob('*.json') if p.is_file())


def print_schema_sample(name: str, items: list, *, limit: int = 2) -> None:
    print(f'\n{name}:')
    print('  count:', len(items))
    for i, it in enumerate(items[:limit]):
        if is_dataclass(it):
            print('  sample', i, asdict(it))
        else:
            print('  sample', i, it)


def validate_items(name: str, items: list, validator) -> None:
    issues = []
    for idx, it in enumerate(items):
        try:
            validator(it)
        except Exception as e:
            issues.append((idx, type(e).__name__, str(e)))
            if len(issues) >= 3:
                break
    if not issues:
        print(f'- {name} schema: OK')
    else:
        print(f'- {name} schema: FAIL (showing first {len(issues)})')
        for idx, et, msg in issues:
            print('  ', idx, et, msg)


def _validate_lat_lon(lat: float, lon: float) -> None:
    if not (-90 <= float(lat) <= 90):
        raise ValueError(f'lat out of range: {lat}')
    if not (-180 <= float(lon) <= 180):
        raise ValueError(f'lon out of range: {lon}')


def validate_bus_stop(x) -> None:
    if not x.stop_uid:
        raise ValueError('missing stop_uid')
    if not x.name:
        raise ValueError('missing name')
    _validate_lat_lon(x.lat, x.lon)


def validate_bike_station(x) -> None:
    if not x.station_uid:
        raise ValueError('missing station_uid')
    if not x.name:
        raise ValueError('missing name')
    _validate_lat_lon(x.lat, x.lon)
    if x.available_rent_bikes is not None and int(x.available_rent_bikes) < 0:
        raise ValueError('available_rent_bikes < 0')
    if x.available_return_bikes is not None and int(x.available_return_bikes) < 0:
        raise ValueError('available_return_bikes < 0')


def validate_metro_station(x) -> None:
    if not x.station_uid:
        raise ValueError('missing station_uid')
    if not x.name:
        raise ValueError('missing name')
    if not x.operator:
        raise ValueError('missing operator')
    _validate_lat_lon(x.lat, x.lon)


def validate_parking_lot(x) -> None:
    if not x.parking_lot_uid:
        raise ValueError('missing parking_lot_uid')
    if not x.name:
        raise ValueError('missing name')
    _validate_lat_lon(x.lat, x.lon)
    if x.available_spaces is not None and int(x.available_spaces) < 0:
        raise ValueError('available_spaces < 0')
    if x.total_spaces is not None and int(x.total_spaces) < 0:
        raise ValueError('total_spaces < 0')


print('TDX cache files before:', count_cache_files())


In [ ]:
# Bus stops
try:
    bus_stops = tdx.get_bus_stops_sample(top=10)
    print('Bus stops:', len(bus_stops))
    print('Example:', bus_stops[0] if bus_stops else None)
    print_schema_sample('bus_stops_sample', bus_stops)
    validate_items('bus_stops_sample', bus_stops, validate_bus_stop)
except Exception as e:
    print('Bus stops unavailable:', type(e).__name__, str(e))


In [ ]:
# YouBike
try:
    bike = tdx.get_youbike_station_statuses_sample(top=10)
    print('Bike stations:', len(bike))
    print('Example:', bike[0] if bike else None)
    print_schema_sample('youbike_sample', bike)
    validate_items('youbike_sample', bike, validate_bike_station)
except Exception as e:
    print('YouBike unavailable:', type(e).__name__, str(e))


In [ ]:
# Metro
try:
    metro = tdx.get_metro_stations_sample(top=10)
    print('Metro stations:', len(metro))
    print('Example:', metro[0] if metro else None)
    print_schema_sample('metro_sample', metro)
    validate_items('metro_sample', metro, validate_metro_station)
except Exception as e:
    print('Metro unavailable:', type(e).__name__, str(e))


In [ ]:
# Parking (may be empty depending on dataset availability for the selected city)
try:
    parking = tdx.get_parking_lot_statuses_sample(top=10)
    print('Parking lots:', len(parking))
    print('Example:', parking[0] if parking else None)
    if parking:
        print_schema_sample('parking_sample', parking)
        validate_items('parking_sample', parking, validate_parking_lot)
except Exception as e:
    print('Parking unavailable (this can be normal):', type(e).__name__, str(e))


In [ ]:
# Cache verification: re-run samples and ensure we don't create new cache entries.
before = count_cache_files()
try:
    _ = tdx.get_bus_stops_sample(top=10)
    _ = tdx.get_youbike_station_statuses_sample(top=10)
    _ = tdx.get_metro_stations_sample(top=10)
    try:
        _ = tdx.get_parking_lot_statuses_sample(top=10)
    except Exception:
        pass
finally:
    after = count_cache_files()

print('TDX cache files after re-run:', after)
print('cache delta:', after - before)
